## Connect with Google Drive

In [ ]:
!pip install gdown

In [ ]:
import gdown

url = "https://drive.google.com/uc?id=1RhakmYA0l99TEQLbHan9rfEyj12hPYL8"

output = 'video.mp4'

gdown.download(url, output)

In [ ]:
SOURCE_VIDEO_PATH = "/kaggle/working/video.mp4"

## Check GPU access

In [ ]:
!nvidia-smi

In [ ]:
import os
HOME = os.getcwd()
print(HOME)

## Install YOLOv8

In [ ]:
!pip install ultralytics==8.0.196

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

In [ ]:
from ultralytics import YOLO

from IPython.display import display, Image

## Preparing a custom dataset

In [ ]:
!pip install --upgrade pip

In [ ]:
!pip install --upgrade jupyter-lsp packaging shapely

In [ ]:
!mkdir {HOME}/datasets
%cd {HOME}/datasets

!pip install roboflow --quiet

from roboflow import Roboflow
rf = Roboflow(api_key="FMJblBkjpLkjWDc3Sr3N")
project = rf.workspace("fyp-fqnbg").project("socket_detection")
dataset = project.version(4).download("yolov8")


## Custom Training

In [ ]:
%cd {HOME}

!yolo task=detect mode=train model=yolov8l.pt data={dataset.location}/data.yaml epochs=100 imgsz=800 plots=True

In [ ]:
!ls {HOME}/runs/detect/train/

In [ ]:
%cd {HOME}
Image(filename=f'{HOME}/runs/detect/train/confusion_matrix.png', width=600)

In [ ]:
%cd {HOME}
Image(filename=f'{HOME}/runs/detect/train/results.png', width=600)

In [ ]:
%cd {HOME}
Image(filename=f'{HOME}/runs/detect/train/val_batch0_pred.jpg', width=600)

## Validate Custom Model

In [ ]:
%cd {HOME}

!yolo task=detect mode=val model={HOME}/runs/detect/train/weights/best.pt data={dataset.location}/data.yaml

## Inference with Custom Model

In [ ]:
%cd {HOME}
!yolo task=detect mode=predict model={HOME}/runs/detect/train/weights/best.pt conf=0.25 source={dataset.location}/test/images save=True

In [ ]:
import glob
from IPython.display import Image, display

for image_path in glob.glob(f'{HOME}/runs/detect/predict/*.jpg')[:3]:
      display(Image(filename=image_path, width=600))
      print("\n")

## Save Files

In [ ]:
%cd {HOME}/runs/detect/train/
!ls

In [ ]:
from tqdm import tqdm
import zipfile

def zip_dir(directory, zip_filename):
    files = os.listdir(directory)
    
    progress = tqdm(total=len(files), desc="Zipping")
    
    with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
        
        for file in files:
            zipf.write(os.path.join(directory, file), arcname=file)
            progress.update(1)
    progress.close()

directory_to_zip = '/kaggle/working/runs/detect/train/'
zip_filename = '/kaggle/working/train_out.zip'

zip_dir(directory_to_zip, zip_filename)

directory_to_zip_2 = '/kaggle/working/runs/detect/train/weights/'
zip_filename_2 = '/kaggle/working/train_weights_out.zip'

zip_dir(directory_to_zip_2, zip_filename_2)

In [ ]:
%cd {HOME}

In [ ]:
from IPython.display import FileLink
FileLink(r'train_out.zip')

In [ ]:
FileLink(r'train_weights_out.zip')